In [12]:
import MDAnalysis as mda
import MDAnalysisData

yiip = MDAnalysisData.yiip_equilibrium.fetch_yiip_equilibrium_short()
u = mda.Universe(yiip.topology, 5 * [yiip.trajectory], in_memory=True)
positions = u.trajectory.get_array()


In [20]:
import numpy as np
import zarr
import h5py

zarr_compressors = {'blosc': (0,9), 'zlib': (0, 9), 'bz2': (1,9), 'lzma': (0,9)}
# For lzma, pass in compression opts as dict(preset=<int>)

hdf5_compressors = {'gzip':(0,9)}


# Load yiip data into minimal zarr file and hdf5 files of different compressors
def create_zarr_traj(compression, compression_opts):
    root = zarr.open(f'yiip_{compression}_{compression_opts}_zarr.zarr', mode='a')
    if compression == 'lzma':
            compression_opts = dict(preset=compression_opts)
    root.create_dataset('positions', data=positions, 
                        chunks = (1, np.shape(positions)[1], 3), 
                        compression=compression, compression_opts=compression_opts)
    
def create_hdf5_traj(compression, compression_opts):
    with h5py.File(f'yiip_{compression}_{compression_opts}_hdf5.h5', 'w') as root:
        root.create_dataset('positions', data=positions, 
                            chunks = (1, np.shape(positions)[1], 3), 
                            compression=compression, compression_opts=compression_opts)

for k, v in zarr_compressors:
    compression = k
    for i in range(v[0], v[1]):
        create_zarr_traj(compression, i)

for k, v in hdf5_compressors:
    compression = k
    for i in range(v[0], v[1]):
        create_zarr_traj(compression, i)



ValueError: too many values to unpack (expected 2)

In [19]:
np.shape(positions)[1]

111815